<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/eras/eras/tune/programs/referee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#共通設定

In [2]:
%tensorflow_version 2.x

##スクラッチ

In [ ]:
keras.backend.clear_session()
gc.collect()

##import

In [3]:
# file dealing
import os
import shutil
from google.colab import drive
# data dealing
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
# process deasing
import gc

# machine learning (back)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, initializers, callbacks

# machine learning
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical


## データフォーマットを規定

In [4]:
class video_format:
  name = "video_format"
  # サンプリングされたCMデータの仕様
  playtime = "15秒"
  displaysize = "(any, any, RGB)"
  videoformat = "any"
  # モデルが扱うCMデータ(上のようなデータは、下のように変換される)
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLORinfo = 3 # "RGB"
  FPS = "2 (FRAME_SIZE / playtime)" # 定義ではなく上から導かれた値

format1 = video_format()

#データをインポート

In [5]:
#@title gdrive マウント
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Already confirm")


Mounted at /content/drive


In [6]:
#@title 一時的にファイルを置くディレクトリ'desk'を作成
desk = '/content/desk'
if not os.path.exists(desk):
  os.mkdir(desk)
os.chdir(desk)

In [7]:
#@title 必要なファイルを'desk'にコピー
%%time
wants_paths = [
'/content/drive/MyDrive/colab/dence1223/normalTrainLabels.npz',
'/content/drive/MyDrive/colab/dence1223/normalTrainVideos.npz',
'/content/drive/MyDrive/colab/dence1223/normalTestLabels.npz',
'/content/drive/MyDrive/colab/dence1223/normalTestVideos.npz'
]

for want in wants_paths:
  if not os.path.exists(os.path.join(desk, os.path.basename(want))):
    shutil.copy2(want, desk)
    print("get : ", want)
            

get :  /content/drive/MyDrive/colab/dence1223/normalTrainLabels.npz
get :  /content/drive/MyDrive/colab/dence1223/normalTrainVideos.npz
get :  /content/drive/MyDrive/colab/dence1223/normalTestLabels.npz
get :  /content/drive/MyDrive/colab/dence1223/normalTestVideos.npz
CPU times: user 68.8 ms, sys: 479 ms, total: 547 ms
Wall time: 12.2 s


In [8]:
def prepare_data():
  # traindata
  v = np.load('/content/desk/normalTrainVideos.npz')
  l = np.load('/content/desk/normalTrainLabels.npz')

  train_videos = [] 
  train_labels = []
  for i in v.files:
    train_videos.append(v[i])
  for i in l.files:
    train_labels.append(l[i])

  train_videos = np.array(train_videos)
  train_labels = np.array(train_labels)
  train_labels = tf.keras.utils.to_categorical(train_labels, 4)

  # testdata
  v = np.load('/content/desk/normalTestVideos.npz')
  l = np.load('/content/desk/normalTestLabels.npz')

  test_videos = []
  test_labels = []
  for i in v.files:
    test_videos.append(v[i])
  for i in l.files:
    test_labels.append(l[i])

  test_videos = np.array(test_videos)
  test_labels = np.array(test_labels)
  test_labels = tf.keras.utils.to_categorical(test_labels, 4)
  
  return train_videos, train_labels,  test_videos, test_labels

In [9]:
train_videos, train_labels,  test_videos, test_labels = prepare_data()

#モデル作成

In [10]:
# モデル作成
model = models.Sequential()
# 入力層
model.add(
     layers.Reshape(
        (format1.FRAME_SIZE,
         format1.HEIGHT,
         format1.WIDTH,
         format1.COLORinfo),
        input_shape=(format1.FRAME_SIZE * format1.HEIGHT * format1.WIDTH * format1.COLORinfo,),
        name='ENTRANCE' )
)
# 中間層
model.add(
    layers.Flatten(name='RIVER' ),
)
model.add(
    layers.Dense(1024,
      activation='relu',
      name='DAM' ),
)
# 出力層
model.add(
    layers.Dense(4, activation='softmax', name='WATERSUPPLY')
)

# 最適化
adam = keras.optimizers.Adam(
    lr=1e-5, beta_1=0.9, beta_2=0.999,
    epsilon=None, decay=0.0, amsgrad=False)
# 誤差関数
mse = tf.keras.losses.MeanSquaredError()
cce = tf.keras.losses.CategoricalCrossentropy()

model.compile(
    optimizer=adam,
    loss=mse,
    metrics=['acc', cce])

#学習実行

In [11]:
def plot(history):
  HEIGHT = 2
  WIDTH = 2

  # 下地の用意
  fig = plt.figure()
  LOSS = fig.add_subplot(HEIGHT, WIDTH, 1)
  ACC = fig.add_subplot(HEIGHT, WIDTH, 2)
  LOSSDIF = fig.add_subplot(HEIGHT, WIDTH, 3)
  plt.subplots_adjust(left=None, bottom=None, right=1.5, top=1.5, wspace=None, hspace=None)

  # 1,1 loss
  loss = DataFrame(history.history['loss'])
  val_loss = DataFrame(history.history['val_loss'])
  loss_props = {
        'title' : 'LOSS',
        'xlabel' : 'epoch',
        'ylabel' : 'value'
    }
  LOSS.set(**loss_props)
  LOSS.plot(loss, label='loss', marker='o', color='blue')
  LOSS.plot(val_loss, label='val_loss', marker='o', linestyle='--', color='orange')

  # 1,2 acc
  acc = DataFrame(history.history['acc'])
  val_acc = DataFrame(history.history['val_acc'])
  acc_props = {
        'title' : 'ACCURACY',
        'xlabel' : 'epoch',
        'ylabel' : 'value'
    }
  ACC.set(**acc_props)
  ACC.plot(acc, label='acc', marker='o', color='blue')
  ACC.plot(val_acc, label='val_acc', marker='o', linestyle='--', color='orange')

  #2,1 loss.diff
  LOSSDIF.plot(loss.diff())
  LOSSDIF.plot(val_loss.diff())
  

  #save
  name = 'tmp'
  path_image = os.path.join(desk, name)
  fig.savefig(path_image)


In [ ]:
# 実行
history = model.fit(
      train_videos, train_labels,
      validation_data=(test_videos, test_labels),
      batch_size=16,
      epochs=512,
      # verbose=0
      )

print("Complete.")
# プロット
plot(history)

Epoch 1/512
9/9 [==============================] - 3s 142ms/step - loss: 0.2550 - acc: 0.4513 - categorical_crossentropy: 14.5869 - val_loss: 0.2742 - val_acc: 0.4516 - val_categorical_crossentropy: 33.0939
Epoch 2/512
9/9 [==============================] - 1s 92ms/step - loss: 0.2830 - acc: 0.4341 - categorical_crossentropy: 35.7954 - val_loss: 0.2742 - val_acc: 0.4516 - val_categorical_crossentropy: 41.0806
Epoch 3/512
9/9 [==============================] - 1s 93ms/step - loss: 0.2877 - acc: 0.4246 - categorical_crossentropy: 47.1236 - val_loss: 0.2742 - val_acc: 0.4516 - val_categorical_crossentropy: 44.3560
Epoch 4/512
9/9 [==============================] - 1s 93ms/step - loss: 0.3081 - acc: 0.3839 - categorical_crossentropy: 51.9052 - val_loss: 0.2742 - val_acc: 0.4516 - val_categorical_crossentropy: 45.7154
Epoch 5/512
9/9 [==============================] - 1s 92ms/step - loss: 0.2827 - acc: 0.4347 - categorical_crossentropy: 45.3815 - val_loss: 0.2742 - val_acc: 0.4516 - val_cat